In [1]:
import numpy as np
import os
import shutil

In [2]:
destination_dir = "/projects/mcmm/results_final/"

In [3]:
# mcir may 16

In [4]:
mcir_dir = "/projects/mcmm/syn_lownodeexp/"

In [5]:
config_copied = []

for filename in os.listdir(mcir_dir):
    if not filename.endswith(".npy"):
        continue
    
    src = os.path.join(mcir_dir, filename)
    
    splitted_fn = filename.split("_")
    splitted_fn.insert(2, "mcir")
    splitted_fn[1] = splitted_fn[1][:-1]
    target_filename = "_".join(splitted_fn)
    dst = os.path.join(destination_dir, target_filename)
    
    config_copied.append((splitted_fn[0].lower(), int(splitted_fn[1])))
    
    shutil.copy2(src, dst)

In [6]:
# mcir may 14

In [7]:
mcir_dirs = [
    "/projects/mcmm/benchmark_2023-05-13/Michalewicz_May12/",
    "/projects/mcmm/benchmark_2023-05-13/Levy_May12/",
    "/projects/mcmm/benchmark_2023-05-13/Ackley_May12/",
    "/projects/mcmm/benchmark_2023-05-13/nlp/",
]

In [8]:
for mcir_dir in mcir_dirs:
    print("Processing {}".format(mcir_dir))
    for filename in os.listdir(mcir_dir):
        if not filename.endswith(".npy"):
            continue
        
        filename_no_appendix = filename.strip(".npy")
        filename_splitted = filename_no_appendix.split("_")
        func = filename_splitted[0]
        dim = int(filename_splitted[1][:-1])
        config = (func.lower(), dim)
        if config in config_copied:
            continue
        
        target_filename_list = [func, str(dim), "mcir", *filename_splitted[2:], "Y"]
        target_filename = "_".join(target_filename_list) + ".npy"
        
        records_Y = np.load(os.path.join(mcir_dir, filename))[:, 0]
        np.save(os.path.join(destination_dir, target_filename), records_Y)

Processing /projects/mcmm/benchmark_2023-05-13/Michalewicz_May12/
Processing /projects/mcmm/benchmark_2023-05-13/Levy_May12/
Processing /projects/mcmm/benchmark_2023-05-13/Ackley_May12/
Processing /projects/mcmm/benchmark_2023-05-13/nlp/


In [9]:
# baselines

In [10]:
baseline_dir_1 = "/projects/mcmm/running_zhizhen_exps/baseline_results_displacement/record"
baseline_dir_2 = "/projects/mcmm/running_zhizhen_exps/baseline_results_displacement_more_iterations/record"

In [11]:
result_files_1 = [fn for fn in os.listdir(baseline_dir_1) if fn.endswith("Y.npy")]
result_files_2 = [fn for fn in os.listdir(baseline_dir_2) if fn.endswith("Y.npy")]

In [12]:
files_in_both = []

for file_1 in result_files_1:
    if file_1 not in result_files_2:
        shutil.copy2(
            os.path.join(baseline_dir_1, file_1),
            os.path.join(destination_dir, file_1)
        )
    else:
        files_in_both.append(file_1)
    
for file_2 in result_files_2:
    if file_2 not in result_files_1:
        shutil.copy2(
            os.path.join(baseline_dir_2, file_2),
            os.path.join(destination_dir, file_2)
        )

In [13]:
for file in files_in_both:
    path_1 = os.path.join(baseline_dir_1, file)
    path_2 = os.path.join(baseline_dir_2, file)
    
    array_1 = np.load(path_1)
    array_2 = np.load(path_2)
    
    assert len(array_1.shape) == 1 and len(array_2.shape) == 1
    
    if len(array_1) > len(array_2):
        src = path_1
    else:
        src = path_2
    
    dst = os.path.join(destination_dir, file)
    
    shutil.copy2(src, dst)

In [14]:
# nlp baselines

In [15]:
baseline_dir_nlp = "/projects/mcmm/baselines_nlp/record/"

In [16]:
result_files = [fn for fn in os.listdir(baseline_dir_nlp) if fn.endswith("Y.npy")]

In [18]:
for file in result_files:
    shutil.copy2(
        os.path.join(baseline_dir_nlp, file),
        os.path.join(destination_dir, file)
    )